### Dependencies installation



In [2]:
import transformers
import numpy as np

In [3]:
#variables to change the model behaviour
squad_v2 = True
batch_size = 16

In [4]:
from datasets import load_dataset, load_metric

In [5]:
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /home/surenis/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from src import vizualize

In [7]:
vizualize.show_random_elements(datasets['train'], 3)

,id,title,context,question,answers
0,573106b7497a881900248b0b,Great_power,"Of the five original great powers recognised at the Congress of Vienna, only France and the United Kingdom have maintained that status continuously to the present day, although France was defeated in the Franco-Prussian War and occupied during World War II. After the Congress of Vienna, the British Empire emerged as the pre-eminent power, due to its navy and the extent of its territories, which signalled the beginning of the Pax Britannica and of the Great Game between the UK and Russia. The balance of power between the Great Powers became a major influence in European politics, prompting Otto von Bismarck to say ""All politics reduces itself to this formula: try to be one of three, as long as the world is governed by the unstable equilibrium of five great powers.""","Who stated that ""All politics reduces itself to this formula: try to be one of three, as long as the world is governed by the unstable equilibrium of five great powers.?""","{'text': ['Otto von Bismarck'], 'answer_start': [596]}"
1,5a819d8531013a001a334ce1,Cotton,"Cotton is used to make a number of textile products. These include terrycloth for highly absorbent bath towels and robes; denim for blue jeans; cambric, popularly used in the manufacture of blue work shirts (from which we get the term ""blue-collar""); and corduroy, seersucker, and cotton twill. Socks, underwear, and most T-shirts are made from cotton. Bed sheets often are made from cotton. Cotton also is used to make yarn used in crochet and knitting. Fabric also can be made from recycled or recovered cotton that otherwise would be thrown away during the spinning, weaving, or cutting process. While many fabrics are made completely of cotton, some materials blend cotton with other fibers, including rayon and synthetic fibers such as polyester. It can either be used in knitted or woven fabrics, as it can be blended with elastine to make a stretchier thread for knitted fabrics, and apparel such as stretch jeans.",For what are jeans used to make?,"{'text': [], 'answer_start': []}"
2,57314befe6313a140071cde4,Mosaic,"In 1913 the Zliten mosaic, a Roman mosaic famous for its many scenes from gladiatorial contests, hunting and everyday life, was discovered in the Libyan town of Zliten. In 2000 archaeologists working in Leptis Magna, Libya, uncovered a 30 ft length of five colorful mosaics created during the 1st or 2nd century AD. The mosaics show a warrior in combat with a deer, four young men wrestling a wild bull to the ground, and a gladiator resting in a state of fatigue, staring at his slain opponent. The mosaics decorated the walls of a cold plunge pool in a bath house within a Roman villa. The gladiator mosaic is noted by scholars as one of the finest examples of mosaic art ever seen — a ""masterpiece comparable in quality with the Alexander Mosaic in Pompeii.""",The mosaics uncovered in Libya were from which century?,"{'text': ['1st or 2nd century AD'], 'answer_start': [293]}"


### Pré-processing

In [8]:
#tokenizer is a global variable thus writing in uppercases
from src.utils import TOKENIZER, MODEL_CHECKPOINT, MAX_LENGTH, DOC_STRIDE

2021-11-23 21:44:21.007565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-23 21:44:21.007610: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Preparing train & computing features

In [20]:
from src import utils

In [14]:
with open("data/trainer_logits.npy", 'rb') as f:
    start_logits = np.load(f)
    end_logits = np.load(f)

In [21]:
features = utils.prepare_train_features(datasets['train'][:5])

In [22]:
tokenized_datasets = utils.tokenize_dataset(datasets)

  0%|          | 0/131 [00:00<?, ?ba/s]

KeyboardInterrupt: 

### Evaluation

In [25]:
validation_features = datasets["validation"].map(
    utils.prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

  0%|          | 0/12 [00:00<?, ?ba/s]

In [31]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [28]:
n_best_size = 20
max_answer_length = 30

In [29]:
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 11.039828, 'text': 'France'},
 {'score': 6.906216,
  'text': 'French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': 6.5083785,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway'},
 {'score': 6.282236, 'text': 'region in France'},
 {'score': 5.6213455, 'text': 'Normandy, a region in France'},
 {'score': 5.3742485, 'text': 'a region in France'},
 {'score': 4.710259,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland'},
 {'score': 4.487678,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark'},
 {'score': 3.919103,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman"'},
 {'score': 3.7067766,
  'text': 'Normanni) were the people who in th

In [30]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [33]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

In [37]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

NameError: name 'final_predictions' is not defined